Справочная информация:
python --version >= 3.10

In [ ]:
# Импорты необходимых библиотек
import os
import logging
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import easyocr
from ultralytics import YOLO

In [ ]:
# Установка уровня логирования (чтобы не выводились сообщения от YOLO - при желании можно не выполнять данный код, но тогда будут видны лишние логи)
logging.getLogger('ultralytics').setLevel(logging.WARNING)

In [ ]:
def ocr_detect(ocr_model : easyocr.Reader, imgs_list):
    """  """
    results = []
    for img in imgs_list:
        results.append(ocr_model.readtext(img))
    
    return results

In [ ]:
# ДИРЕКТОРИЯ, ГДЕ НАХОДИТСЯ ПАПКА
INPUT_PATH = r"..\test Росатом\test\imgs" # <-- FIXME: ИЗМЕНИТЕ ДАННУЮ ДИРЕКТОРИЮ НА ДИРЕКТОРИЮ ВАЛИДАЦИОННОГО ДАТАСЕТА
OUTPUT_PATH = r".\outputs"               # <-- FIXME: ИЗМЕНИТЕ НА ДИРЕКТОРИЮ В КОТОРУЮ НЕОБХОДИМО СОХРАНИТЬ ФАЙЛЫ

INPUT_PATH = INPUT_PATH.replace("/", "\\")
OUTPUT_PATH = OUTPUT_PATH.replace("/", "\\")

In [ ]:
# получение списка путей до файлов в директории валидационного датасета
IMG_FILES = os.listdir(INPUT_PATH)
IMG_FILES[:5]

In [ ]:
# ГЛОБАЛЬНЫЕ ПЕРЕМЕННЫЕ

YOLO_MODEL_PATH = r".\models\winstrike_yolo11.pt" # <-- FIXME: ПЕРЕД НАЧАЛОМ ДЕТЕКЦИИ ПРОИЗВЕДИТЕ СКАЧИВАНИЕ ВЕСОВ ДЛЯ МОДЕЛИ YOLO ПО ССЫЛКЕ: https://disk.yandex.ru/d/3TM36EKEcFw3uQ
                                                  # ПОСЛЕ СКАЧИВАНИЯ ФАЙЛА *.pt С ВЕСАМИ ПОМЕСТИТЕ ЕГО В ПАПКУ "./models/" 

EASYOCR_CONFIG = {
    "lang_list": ['en', 'ru'],
    "gpu": True                          # <-- FIXME: ИЗМЕНИТЕ ПАРАМЕТР "gpu" на False В СЛУЧАЕ ОТСУТСТВИЯ ВОЗМОЖНОСТИ ЗАПУСКА НА GPU (только CPU)
}

In [ ]:
# Создаем объекты моделей нейронных сетей, производящих локализацию области с текстом и перевод текста, представленного в данной области в строку
YOLO_MODEL = YOLO(YOLO_MODEL_PATH)

EASYOCR_MODEL = easyocr.Reader(**EASYOCR_CONFIG)

In [ ]:
# СОЗДАНИЕ ОБЪЕКТА DATAFRAME С ИНФОРМАЦИЕЙ О РАСПОЗНАННЫХ ОБЪЕКТАХ
detected_info_df = {
    "image_file" : [],
    "label" : [],
    "label_text" : []
}

detected_info_df = pd.DataFrame(detected_info_df)
detected_info_df

In [ ]:
# ДЕТЕКЦИЯ ОБЪЕКТОВ 
for image_file in IMG_FILES:

    cur_row_to_df = []

    start_time = time.time()
    if not image_file.endswith(('.png', '.jpg', '.jpeg', '.JPG')):
        continue
    else:
        try:
            cur_row_to_df.append(image_file)
            
            print('-------------------------------------')
            print(f'Название изображения : {image_file}')
            # Создаем путь до изображения
            image_file_path = os.path.join(INPUT_PATH, image_file)

            # Загружаем изображение
            image = cv2.imread(image_file_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Выполняем детекцию
            results = YOLO_MODEL(image_rgb)

            # Копируем изображение для отрисовки
            image_copy = image_rgb.copy()

            text_imgs = []
            coords_text = []

            # Проходимся по результатам детекции
            for result in results[0].boxes.data:
                x_min, y_min, x_max, y_max, confidence, class_id = result[:6].int().tolist()

                # сохраняем изображение с текстом
                text_imgs.append(image_copy[y_min:y_max, x_min:x_max])

                # сохраняем координаты
                result = results[0].boxes.xywhn[0]
                coords_text.append(f"0 {result[0]:.6f} {result[1]:.6f} {result[2]:.6f} {result[3]:.6f}\n")

                # Рисуем прямоугольник
                cv2.rectangle(image_copy, (x_min, y_min), (x_max, y_max), (0, 255, 0), 5)


            # СОХРАНЕНИЕ ИЗОБРАЖЕНИЯ
            try:
                cv2.imwrite(os.path.join(OUTPUT_PATH, image_file), image_copy)
            except:
                print(f"[WARNING] не удалось сохранить изображение \"{image_file}\"")

            # УТОЧНЕНИЕ МАРКИРОВКИ
            text_result = []

            for detected_contour_text in ocr_detect(EASYOCR_MODEL, text_imgs):
                for text_detect in detected_contour_text:
                    text_result.append(text_detect[1]) 

            final_text = " ".join(text_result)

            coords_text.append("")
            cur_row_to_df.append(f'{"".join(coords_text)}')
            cur_row_to_df.append(f'\"{final_text}\"')

            print(f'текст на фото : \"{final_text}\"')
            plt.figure(figsize=(5, 5))
            plt.imshow(image_copy)
            plt.axis('off')
            plt.show()


        except Exception as e:
            cur_row_to_df.append("")
            cur_row_to_df.append("")
            print('текст не размечен')

        print(f"Время распознавания - {time.time() - start_time:.3f} мс")
        detected_info_df.loc[len(detected_info_df)] = cur_row_to_df

    

In [ ]:
# ДОБАВЛЕНИЕ ШАБЛОНА ДЛЯ СТРОК, КОТОРЫЕ АЛГОРИТМ НЕ РАСПОЗНАЛ
detected_info_df.loc[detected_info_df['label'].str.len() < 10, 'label'] = '0 0 0 0 0\n'

In [ ]:
# СОРТИРОВКА РЕЗУЛЬТАТОВ
detected_info_df['numeric_part'] = detected_info_df['image_file'].str.extract('(\d+)').astype(int)
detected_info_df = detected_info_df.sort_values(by='numeric_part').drop(columns=['numeric_part'])

In [ ]:
# ПРОСМОТР ПОЛУЧИВШЕЙСЯ ТАБЛИЦЫ
detected_info_df

In [ ]:
# СОХРАНЕНИЕ РЕЗУЛЬТАТА
detected_info_df.to_csv("submission.csv", index=False)

In [ ]:
pd.read_csv("submission.csv")